In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
import requests

In [2]:
url_list = []
for i in range(1, 42):
    url = f"https://www.flipkart.com/search?q=mobiles&as=on&as-show=on&otracker=AS_Query_HistoryAutoSuggest_1_6_na_na_na&otracker1=AS_Query_HistoryAutoSuggest_1_6_na_na_na&as-pos=1&as-type=HISTORY&suggestionId=mobiles&requestId=a4b17236-22a6-4fa3-800b-37acc52f0ff0&as-searchtext=mobile&page={i}"
    url_list.append(url)

In [3]:
def create_html_page(url):
    uClient = urlopen(url)
    page_html = uClient.read()
    uClient.close()
    page_soup = BeautifulSoup(page_html, features='html')
    return page_soup

In [4]:
def create_final_data(page_soup):
    data = {"Product Name": [],
            "Sale & Actual Price": [],
            "Raview & Rattings": [],
            "Offer": [],
            "Camera": [],
            "Memory": [],
            "Battery": [],
            "Display": [],
            "Ratting Number": []}

    all_info_class_name = "yKfJKb row"
    containers = page_soup.find_all("div", {"class": all_info_class_name})
    
    for contain in containers:
        product_name = contain.find_all("div", {"class": "KzDlHZ"})[0].text
        data["Product Name"].append(product_name)

        rr = contain.find_all("span", {"class": "Wphh3N"})
        if len(rr) == 0:
            review_and_ratting = None
            ratting_number = None
        else:
            review_and_ratting = rr[0].text
            ratting_number = contain.find_all("div", {"class": "XQDdHH"})[0].text

        data["Raview & Rattings"].append(review_and_ratting)
        data["Ratting Number"].append(ratting_number)

        specifictations = contain.find_all("div", {"class": "_6NESgJ"})[0]

        data["Memory"].append(specifictations.find_all('li')[0].text)
        data["Display"].append(specifictations.find_all('li')[1].text)
        data["Camera"].append(specifictations.find_all('li')[2].text)
        data["Battery"].append(specifictations.find_all('li')[3].text)

        price = contain.find_all("div", {"class": "hl05eU"})[0].text
        data["Sale & Actual Price"].append(price)

        offer_list = contain.find_all("div", {"class": "UkUFwK"})
        if len(offer_list) == 0:
            data["Offer"].append("No Offer")
        else:
            data["Offer"].append(offer_list[0].text)
        
    return data
    

# Main Section

In [12]:
data_frame_list = []
for i, url in enumerate(url_list):
    print(f"Started url page {i+1}")
    page_soup = create_html_page(url)
    data_info = create_final_data(page_soup)
    data_frame_list.append(data_info)

Started url page 1
Started url page 2
Started url page 3
Started url page 4
Started url page 5
Started url page 6
Started url page 7
Started url page 8
Started url page 9
Started url page 10
Started url page 11


HTTPError: HTTP Error 429: Too Many Requests

In [15]:
len(data_frame_list)

10

In [25]:
df_all = []
for i in range(len(data_frame_list)):
    df = pd.DataFrame(data_frame_list[i])
    df_all.append(df)

In [26]:
final_df = pd.concat(df_all, ignore_index=True)

In [27]:
final_df["Product Name"].unique().shape

(177,)

In [29]:
final_df.shape

(240, 9)

In [31]:
final_df.to_csv("flipkart_mobile_phones.csv")